Use this notebook as the central place to collect all data dumps and process them into figures for _Likelihood plots_.

In [1]:
import os
import sys
import torch
import altair as alt
import pandas as pd
import numpy as np

alt.data_transformers.disable_max_rows();


In [7]:
## TODO: change this to the dump path.
# data_dump_path = '../../swag-3body-data.pt'
data_dump_path = '../../de-3body-data.pt'

assert os.path.isfile(swag_dump_path)
assert os.path.isfile(de_dump_path)

swag_dump = torch.load(swag_dump_path)
de_dump = torch.load(de_dump_path)

ts = data_dump.get('ts')
z0_orig = data_dump.get('z0_orig')
true_zt = data_dump.get('true_zt')
# true_zt_chaos = data_dump.get('true_zt_chaos').permute(1, 0, 2, 3, 4, 5)
de_pred_zt = data_dump.get('pred_zt')

data_dump_path = '../../swag-3body-data.pt'
data_dump = torch.load(data_dump_path)
swag_pred_zt = data_dump.get('pred_zt')

In [28]:
def compute_likelihood(ref, pred):
    '''
    Likelihood of the reference under Gaussian estimated
    by the pred samples, factored over time.
    Arguments:
    ref: N x T x 2 x B x 2
    pred: M x N x T x 2 x B x 2
    '''
    batch_shape = pred.shape[:3]
    
    print(pred.view(*batch_shape, -1).shape)
    
    pred_mu = pred.view(*batch_shape, -1).mean(dim=0)
    pred_std = pred.view(*batch_shape, -1).std(dim=0) + 1e-6
    pred_std[torch.isnan(pred_std)] = 1e-6
    pred_dist = torch.distributions.MultivariateNormal(pred_mu, pred_std.diag_embed())

    log_prob = pred_dist.log_prob(ref.view(*ref.shape[:2], -1)).clamp(-100,100)  ## N x T
    return -1*log_prob


In [29]:
#chaos_likelihood = compute_likelihood(true_zt, true_zt_chaos)
de_likelihood = compute_likelihood(true_zt, de_pred_zt)
swag_likelihood = compute_likelihood(true_zt, swag_pred_zt)

torch.Size([10, 5, 334, 12])
torch.Size([10, 5, 334, 12])


In [32]:
def plot_likelihood(likelihood, color):
    y_mean = likelihood.mean(0).cpu().numpy()
    y_std = likelihood.std(0).cpu().numpy()

    err_mean_chart = alt.Chart(pd.DataFrame({
        't': ts.cpu().numpy(),
        'y': y_mean,
        'y_hi': y_mean + 2. * y_std,
        'y_lo': y_mean - 2. * y_std,
    })).mark_line(color=color, opacity=0.7).encode(x='t', y='y')
    err_std_chart = err_mean_chart.mark_area(color=color, opacity=0.2).encode(
        x='t', y=alt.Y('y_lo', title='NLL'), y2='y_hi')

    return err_mean_chart + err_std_chart


In [33]:
plot_likelihood(de_likelihood, 'darkgreen') + plot_likelihood(swag_likelihood, 'red')

alt.LayerChart(...)